# **IST 652 Mini Project: Cinematic Insights**
- Nikki Shimao

## Background:

The idea for this project emerged from the current buzz around the highly anticipated movie releases, such as Wicked and Moana, which have generated significant excitement. With so much attention focused on these upcoming films, it sparked an interest in exploring movie ratings and understanding what drives people's preferences. I personally love movies and am always curious about other's ratings of them in order to help me pick what films I should watch next!

By analyzing a dataset of movies and user ratings, the project aims to uncover patterns in how films are rated across genres (specifically Drama/Romance), track how ratings evolve over time, and examine specific user's ratings through multiple ways. This analysis of movie ratings offers a data-driven approach to connecting trends in the film industry with a deeper understanding of viewer preferences.

## Part 1: Import and Clean 2 Data Sources

In [37]:
# Import required packages
import pandas as pd

In [38]:
# Read in data from first source:
allmovies_df = pd.read_csv('movies.csv')

In [39]:
# Allows viewing of type of allmovies_df
print(type(allmovies_df))

<class 'pandas.core.frame.DataFrame'>


In [40]:
# Checking the data types of each column
print(allmovies_df.dtypes)

movieId     int64
title      object
genres     object
dtype: object


In [41]:
# Included these lines of code to reset the display options to default so that I can show some changes that I eventually do 
pd.reset_option('display.width')  # Reset display width to default
pd.reset_option('display.max_columns')  # Reset max columns to default

In [42]:
# Output the first few rows of the movies dataset
print(allmovies_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [43]:
# Reformatted the movies dataset so that it would not line break the genres column, but instead kept it on the same lines as the movieId and title

# Set the display width in order to avoid any line breaks
pd.set_option('display.width', 200)

# Ensuring that all columns are displayed
pd.set_option('display.max_columns', None)


In [44]:
# Prints the first few lines of the allmovies_df to show it reformatted
print(allmovies_df.head())

   movieId                               title                                       genres
0        1                    Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1        2                      Jumanji (1995)                   Adventure|Children|Fantasy
2        3             Grumpier Old Men (1995)                               Comedy|Romance
3        4            Waiting to Exhale (1995)                         Comedy|Drama|Romance
4        5  Father of the Bride Part II (1995)                                       Comedy


In [45]:
# Checking for any missing values
print(allmovies_df.isnull().sum())

movieId    0
title      0
genres     0
dtype: int64


In [46]:
# Checking for duplicates
print(allmovies_df.duplicated().sum())

0


In [47]:
# Despite there being no missing values or duplicates there is some cleaning and reconfigurating of the data that I thought was necessary

# Extracts the year from the 'title' column and assigns it to a new column
allmovies_df['year'] = allmovies_df['title'].str.extract(r'\((\d{4})\)')

# Removes the year from the title column
allmovies_df['title'] = allmovies_df['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()

# Replaces the '|' character in the 'genres' column with a comma
allmovies_df['genres'] = allmovies_df['genres'].str.replace('|', ',', regex=False)

# Saves the cleaned data to cleanmovies_df
cleanmovies_df = allmovies_df.copy()

# Prints the first few lines of the cleaned movies dataframe
print(cleanmovies_df.head())


   movieId                        title                                       genres  year
0        1                    Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
1        2                      Jumanji                   Adventure,Children,Fantasy  1995
2        3             Grumpier Old Men                               Comedy,Romance  1995
3        4            Waiting to Exhale                         Comedy,Drama,Romance  1995
4        5  Father of the Bride Part II                                       Comedy  1995


In [48]:
# Checking the data types of each column
print(cleanmovies_df.dtypes)

movieId     int64
title      object
genres     object
year       object
dtype: object


In [49]:
# Read in data from second source:
allratings_df = pd.read_csv('ratings.csv')

In [50]:
# Output the first few rows of the ratings dataset
print(allratings_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [51]:
# Checking the data types of each column
print(allratings_df.dtypes)

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


In [52]:
# Checking for any missing values
print(allratings_df.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [53]:
# Checking if there are duplicate ratings (same user rating the same movie more than once)

# Identifying the duplicate ratings by filtering and grouping
duplicate_ratings = allratings_df.groupby(['userId', 'movieId']).filter(lambda x: len(x) > 1)

# Print any duplicate ratings (hoping it doesn't print any)
print(duplicate_ratings)


Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


In [54]:
# Despite there being no missing values or duplicates there is some cleaning and reformatting of the data that I thought was necessary

# Converts the timestamp column from Unix time to datetime and saving it as a new column
allratings_df['datetime'] = pd.to_datetime(allratings_df['timestamp'], unit='s')

# Saves the cleaned DataFrame to cleanratings_df
cleanratings_df = allratings_df.copy()

# Prints the first few lines of the cleaned ratings dataframe
print(cleanratings_df.head())


   userId  movieId  rating  timestamp            datetime
0       1        1     4.0  964982703 2000-07-30 18:45:03
1       1        3     4.0  964981247 2000-07-30 18:20:47
2       1        6     4.0  964982224 2000-07-30 18:37:04
3       1       47     5.0  964983815 2000-07-30 19:03:35
4       1       50     5.0  964982931 2000-07-30 18:48:51


## Part 2: Join the 2 Data Sources

In [55]:
# Merging the clean movies data source with the clean rating data source

# Matching the rows based on the common column, 'movieId'
merged_df = pd.merge(cleanratings_df, cleanmovies_df, on='movieId', how='inner')

# Checks the results
print(merged_df.head()) # Looking at first 5 movies of the merged dataset
print(merged_df.tail()) # Looking at last 5 movies of the merged dataset


   userId  movieId  rating   timestamp            datetime      title                                       genres  year
0       1        1     4.0   964982703 2000-07-30 18:45:03  Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
1       5        1     4.0   847434962 1996-11-08 06:36:02  Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
2       7        1     4.5  1106635946 2005-01-25 06:52:26  Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
3      15        1     2.5  1510577970 2017-11-13 12:59:30  Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
4      17        1     4.5  1305696483 2011-05-18 05:28:03  Toy Story  Adventure,Animation,Children,Comedy,Fantasy  1995
        userId  movieId  rating   timestamp            datetime                      title                 genres  year
100831     610   160341     2.5  1479545749 2016-11-19 08:55:49                  Bloodmoon        Action,Thriller  1997
100832     610   160527     4.5  1

## Part 3: Analyzing and Summarizing
#### 3 Questions

### Q1: What are the titles of the movies with the highest average ratings that belong exclusively to the Drama/Romance genre?
**Results should be ordered from the earliest to the most recent year**

In [56]:
# Calculates the average rating for each movie
avg_ratings = merged_df.groupby(['movieId', 'title', 'year'])['rating'].mean().reset_index(name='avg_rating')

# Calculates the highest average rating
highest_avg_rating = avg_ratings['avg_rating'].max()

# Filters the dataframe to get all movies with the highest average rating
highest_rated_movies = avg_ratings[avg_ratings['avg_rating'] == highest_avg_rating] # Creates new data frame that contains only the rows where the average rating is equal to the highest average rating 

# Prints the title, year, and average rating of each movie with the highest average rating
print("Movies with the highest average rating:\n")
for index, row in highest_rated_movies.iterrows():
    print(f"Movie: {row['title']} ({row['year']})")
    print(f"Average Rating: {row['avg_rating']:.2f}\n")


Movies with the highest average rating:

Movie: Lamerica (1994)
Average Rating: 5.00

Movie: Heidi Fleiss: Hollywood Madam (1995)
Average Rating: 5.00

Movie: Awfully Big Adventure, An (1995)
Average Rating: 5.00

Movie: Live Nude Girls (1995)
Average Rating: 5.00

Movie: In the Realm of the Senses (Ai no corrida) (1976)
Average Rating: 5.00

Movie: What Happened Was... (1994)
Average Rating: 5.00

Movie: Thin Line Between Love and Hate, A (1996)
Average Rating: 5.00

Movie: Denise Calls Up (1995)
Average Rating: 5.00

Movie: Supercop 2 (Project S) (Chao ji ji hua) (1993)
Average Rating: 5.00

Movie: Entertaining Angels: The Dorothy Day Story (1996)
Average Rating: 5.00

Movie: Lesson Faust (1994)
Average Rating: 5.00

Movie: Hype! (1996)
Average Rating: 5.00

Movie: Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)
Average Rating: 5.00

Movie: Assignment, The (1997)
Average Rating: 5.00

Movie: Four Days in September (O Que É Isso, Companheiro?) (1997)
Average Ratin

In [57]:
# Filters for movies that are in both 'Drama' and 'Romance' genres
filtered_movies = merged_df[merged_df['genres'].str.contains('Drama', case=False, na=False) &
                            merged_df['genres'].str.contains('Romance', case=False, na=False)]

# Excludes movies that contain genres other than 'Drama' and 'Romance'
filtered_movies = filtered_movies[filtered_movies['genres'].apply(lambda x: sorted(x.split(',')) == ['Drama', 'Romance'])]

# Merges with avg_ratings to get the avg_rating column
highest_rated_filtered_movies = filtered_movies.merge(avg_ratings[['movieId', 'avg_rating']], on='movieId', how='left')

# Filters for movies with the highest average rating
highest_rated_filtered_movies = highest_rated_filtered_movies[highest_rated_filtered_movies['avg_rating'] == highest_avg_rating]

# Prints the title, year, average rating, and genre of each movie with the highest average rating and are just Drama/Romance genre
for index, row in highest_rated_filtered_movies.iterrows():
    print(f"Movie: {row['title']} ({row['year']})")
    print(f"Average Rating: {row['avg_rating']:.2f}")
    print(f"Genres: {row['genres']}\n")


Movie: Moscow Does Not Believe in Tears (Moskva slezam ne verit) (1979)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Cruel Romance, A (Zhestokij Romans) (1984)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Man and a Woman, A (Un homme et une femme) (1966)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Mr. Skeffington (1944)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Jane Eyre (1944)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Rain (2001)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Lady Jane (1986)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Sandpiper, The (1965)
Average Rating: 5.00
Genres: Drama,Romance



In [58]:
# Sorts the filtered dataframe from above from oldest to most recent
highest_rated_filtered_movies = highest_rated_filtered_movies.sort_values(by='year', ascending=True)

# Prints the filtered movies from above but from oldest to most recent 
for index, row in highest_rated_filtered_movies.iterrows():
    print(f"Movie: {row['title']} ({row['year']})")
    print(f"Average Rating: {row['avg_rating']:.2f}")
    print(f"Genres: {row['genres']}\n")


Movie: Mr. Skeffington (1944)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Jane Eyre (1944)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Sandpiper, The (1965)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Man and a Woman, A (Un homme et une femme) (1966)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Moscow Does Not Believe in Tears (Moskva slezam ne verit) (1979)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Cruel Romance, A (Zhestokij Romans) (1984)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Lady Jane (1986)
Average Rating: 5.00
Genres: Drama,Romance

Movie: Rain (2001)
Average Rating: 5.00
Genres: Drama,Romance



### Q2: First, how do the first and last ratings of each movie compare? 
#### Specifically, which movies have a rating increase of at least 4 points from the first to the last rating, and what are their titles and corresponding datetimes? 

In [59]:
# Groups the data and retrieves the first and last ratings and first and last datetimes for each movie and its corresponding title 
rating_comparison = merged_df.groupby('movieId').agg(
    first_rating=('rating', 'first'),
    last_rating=('rating', 'last'),
    first_datetime=('datetime', 'first'),
    last_datetime=('datetime', 'last'),
    title=('title', 'first')
).reset_index()

# Displays the comparison of the ratings along with printing the movieId, movie title, and the first and last datetimes
print(rating_comparison[['movieId', 'title', 'first_rating', 'last_rating', 'first_datetime', 'last_datetime']])


      movieId                               title  first_rating  last_rating      first_datetime       last_datetime
0           1                           Toy Story           4.0          5.0 2000-07-30 18:45:03 2016-11-19 08:08:20
1           2                             Jumanji           4.0          2.0 1996-10-17 11:58:42 2005-05-30 22:06:26
2           3                    Grumpier Old Men           4.0          2.0 2000-07-30 18:20:47 2005-05-31 01:53:33
3           4                   Waiting to Exhale           3.0          1.5 1996-10-17 12:12:29 2009-03-22 22:14:15
4           5         Father of the Bride Part II           5.0          3.0 1996-10-17 12:05:38 1996-05-14 13:25:55
...       ...                                 ...           ...          ...                 ...                 ...
9719   193581  Black Butler: Book of the Atlantic           4.0          4.0 2018-09-16 14:44:42 2018-09-16 14:44:42
9720   193583               No Game No Life: Zero           3.5 

#### Dives into the specifics of the movies with an increased rating of at least 4:

In [60]:
# Filter movies where the last rating is 4 or more greater than the first rating
updated_rating_comparison = rating_comparison[rating_comparison['last_rating'] - rating_comparison['first_rating'] >= 4]

# Displays the information of those movies with an increased rating of at least 4
print(updated_rating_comparison[['movieId', 'title', 'first_rating', 'last_rating', 'first_datetime', 'last_datetime']])

      movieId                                            title  first_rating  last_rating      first_datetime       last_datetime
585       720  Wallace & Gromit: The Best of Aardman Animation           0.5          4.5 2011-05-27 02:33:15 2017-06-27 03:23:52
619       784                                   Cable Guy, The           1.0          5.0 1996-06-22 11:20:47 2016-11-19 08:38:43
667       879                                       Relic, The           1.0          5.0 2000-08-08 03:26:35 1997-05-18 17:33:40
696       914                                     My Fair Lady           0.5          5.0 2011-05-27 02:32:47 2007-02-11 21:20:46
911      1211        Wings of Desire (Himmel über Berlin, Der)           1.0          5.0 2000-07-25 15:33:25 2007-02-13 14:18:23
961      1263                                 Deer Hunter, The           0.5          5.0 2011-05-27 02:32:49 2000-03-24 19:41:40
970      1272                                           Patton           0.5          5.0 

### Q3: What are the movie titles and ratings of the movies rated by User 19? (userId = 19)
#### Follow-up questions: 
##### What is User 19's average rating? 
##### How many 5 star ratings has User 19 given and what are the titles of those movies? 
##### What are the titles of the first 50 movies of User 19's lowest rated movies?

In [61]:
# Filters the merged_df and retreives the movie titles and ratings for user 19
user_19_ratings = merged_df[merged_df['userId'] == 19][['title', 'rating']]

# Sorts the data frame by descending order
user_19_ratings_sorted = user_19_ratings.sort_values(by='rating', ascending=False)

# Prints the sorted user 19 data frame
print(user_19_ratings_sorted)

                                                   title  rating
23704                                Defending Your Life     5.0
14612                           Who Framed Roger Rabbit?     5.0
63004  Adventures of Buckaroo Banzai Across the 8th D...     5.0
63794                                     Empire Records     5.0
58832                                           Heathers     5.0
...                                                  ...     ...
5871                                 Alice in Wonderland     1.0
64286                                         Hollow Man     1.0
5043                                           Space Jam     1.0
64331                                        Phantasm II     1.0
64334                              Phantasm IV: Oblivion     1.0

[703 rows x 2 columns]


In [62]:
# Calculates the average of all the ratings given by user 19
user_19_avg_rating = user_19_ratings['rating'].mean()

# Prints User 19's average rating
print(f"User 19's average rating: {user_19_avg_rating}")


User 19's average rating: 2.607396870554765


In [63]:
# Selects only the movies that user 19 has given a 5.0 to
five_star_ratings = user_19_ratings[user_19_ratings['rating'] == 5]

# Prints the results
print(f"User 19 has rated {len(five_star_ratings)} movies with 5 stars.") # Outputs the number of movies User 19 has given a 5.0 to
print(five_star_ratings[['title', 'rating']]) # Outputs all of these movies with their title and rating


User 19 has rated 26 movies with 5 stars.
                                                   title  rating
4128                                              Batman     5.0
6458                          E.T. the Extra-Terrestrial     5.0
6992                                 Princess Bride, The     5.0
7138   Raiders of the Lost Ark (Indiana Jones and the...     5.0
9070                  Indiana Jones and the Last Crusade     5.0
10481                                          Labyrinth     5.0
14330                                         Fight Club     5.0
14612                           Who Framed Roger Rabbit?     5.0
20487                                        Rear Window     5.0
23704                                Defending Your Life     5.0
26721                         Terminator 2: Judgment Day     5.0
31216                                          Crow, The     5.0
45704                                 Fifth Element, The     5.0
48718                                Maltese Fal

In [64]:
# Sorts the data frame by ascending order then selects the 50 movies with the lowest ratings given by User 19
lowest_rated_by_user_19 = user_19_ratings.sort_values(by='rating').head(50)

# Prints the title and rating of User 19's lowest rated movies
print(lowest_rated_by_user_19[['title', 'rating']])

                                               title  rating
64334                          Phantasm IV: Oblivion     1.0
64331                                    Phantasm II     1.0
64286                                     Hollow Man     1.0
63115                               Fatal Attraction     1.0
63188                     Deuce Bigalow: Male Gigolo     1.0
63205                                      Supernova     1.0
63251                            Eye of the Beholder     1.0
62496                               Inspector Gadget     1.0
62587                               Deadtime Stories     1.0
62066                                  Deadly Friend     1.0
62112                                   Pet Sematary     1.0
62192                                  Baby Geniuses     1.0
62262                Chopping Mall (a.k.a. Killbots)     1.0
62275                                   Superman III     1.0
62297               Superman IV: The Quest for Peace     1.0
61671                   

## Part 4: Outputs and Summary

Here are the resulting outputs from each of the questions above

In [65]:
# Q1 results output
highest_rated_filtered_movies.to_csv('highest_avg_rated_drama_romance.csv', index=False)

In [66]:
# Q2 results output
updated_rating_comparison.to_csv('rating_improve_4_more.csv', index=False)

In [67]:
# Q3 results output
user_19_ratings_sorted.to_csv('user_19_ratings_sorted.csv', index=False)

In conclusion, by analyzing the highest-rated Drama/Romance films, tracking how ratings evolve over time, and examining individual user preferences, this project offers practical insights that can help viewers find highly recommended films. Whether you're seeking critically acclaimed movies or curious about how certain films are perceived over time, the outputs from this project make it easier to pick a movie that aligns with your tastes and rating trends.